# Prerequisites

In [ ]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [ ]:
data = pd.read_csv('today.csv')

## Cleaning the File

### Date Formatting

In [ ]:
cols = ['Date', 'Shipment Month', 'Terminal Month', 'M2M Terminal Month']
for i in cols:
    formatted_dates = pd.to_datetime(data[i], format='mixed', errors='coerce')
    data[i] = formatted_dates

# Making Pivot Tables

## Pivots for Groups Of Units

In [ ]:
group = input("Enter the units you want to group (comma separated): ")
group_list = group.split(',')
group = [i.strip() for i in group_list]
print(f"Grouping the following units: {group}")
group_data = data[data['Unit'].isin(group_list)]
name = input("Enter a name for the group: ")
os.makedirs(f"Group Files/{name}", exist_ok=True)
group_data.to_csv(f"Group Files/{name}/unit_data_today.csv", index=False)

### Outright for Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[~unit_data['Trade Type'].isin(['FX','FX_BK'])]
pivot_table = filter_data.pivot_table(
    columns='M2M Terminal Month',
    index='Trade Type',
    values='Qty Exposure',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
outright_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

### Basis Position for Each Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[unit_data['Transaction Type'].isin(['PHY'])]
pivot_table = filter_data.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='Delta Quantity',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
basis_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

### M2M Currency Risk for Each Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[unit_data['Transaction Type'].isin(['PHY'])]
pivot_table = filter_data.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='M2M Currency Risk',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
m2mcurr_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

### Euro Exposure for Each Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")

# Filter data for Euro Exposure
filter_data = unit_data[(unit_data['Contract Currency'] == 'EUR') & (unit_data['Transaction Type'] == 'PHY')]
filter_data2 = unit_data[(unit_data['Exch'] == 'EOP') & (unit_data['Trade Type'].isin(['FUT', 'CALL', 'PUT']))]
filter_data3 = unit_data[(unit_data['Contract Currency'] == 'EUR') & (unit_data['Trade Type']== 'FX')]
combined = pd.concat([filter_data, filter_data2, filter_data3], ignore_index=True)
combined_unique = combined.drop_duplicates()
combined_unique["Exposure"] = filter_data['Delta Quantity'] * filter_data['Price'] * -1


pivot_table = combined_unique.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='Exposure',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
eurexp_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

In [10]:
wb = openpyxl.Workbook()
excel_path = f"Group Files/{name}/Position_Pivots.xlsx"

with pd.ExcelWriter(excel_path,engine='openpyxl') as writer:

    outright_pivot_table.to_excel(writer,sheet_name='Outright Position')
    basis_pivot_table.to_excel(writer,sheet_name='Basis Position')
    m2mcurr_pivot_table.to_excel(writer,sheet_name='M2M Currency Risk Position')
    eurexp_pivot_table.to_excel(writer,sheet_name='Euro Exposure Position')